# Sreality
### 12.3.2020

## AUTOMATIZACE
### 13.4.2020


In [331]:
##Poznámky:

# df.isna().any() vyhodí seznam sloupců, ve kterém je NAN hodnota
#data.isin([-1]).any()   mi najde -1 kdekoliv v df
# df.dropna()
# df.fillna(0)

# data[data.duplicated(subset='url_id', keep=False)].sort_values(by=['url_id'])    = Hledání duplikátů


#data_all.plocha.value_counts()
#data.groupby("plocha_2")["popis"].count()
# Tohle mi to řadí i podle počtu, což ale nuztně nechci, groupby mi to hodí spíš podle abecedy, tím najdu divné hodnoty
# Ještě by byl ořešení přes nunique(), ale to it neřekne které to jsou, jen ukáže celkový počet druhů



#PRoblém !!!! U satahování PRONÁJEM BYTY mi to správně detekuje počet inzerátů a stran ,ale ukládání URLs skončí po první stránce !! jen20 záznamů !!!


# Obsah

1. Importování Packagů
2. Definice všech funkcí
3. Spuštění Procesu
4. Analýzy a grafy
5. Export do HTML a zaslání na mail

-----------------------
# 1. Importování Packagů
-----------------------

In [1]:

##### Obecné ############
import pandas as pd # for dataframe
from pandas import DataFrame
import numpy as np # for arrays
import matplotlib as plt
from matplotlib.pyplot import figure     # for saving and changing size of plots

from collections import Counter             # Na počítání prvků v dimenzi  
from datetime import datetime
import re                    # relativní Novinka - regular expressions
from time import sleep # for sleeping (slowing down) inside a function
import random  # for random number sleeping
import math    # Zaokrouhlování
import time    # Měření scrappingu délka spánků
import datetime  # Datum do zapsání do Adresáře

import itertools   # Tool na odlistování vnořených listů



##### David a Scraping ############
import requests # for robots check
from bs4 import BeautifulSoup
from selenium import webdriver  

##### Lukáš a GeoPy ############        # Odsud dolů samé novinky:
from geopy.geocoders import Nominatim   # Geolocator   # pip install geopy  # Co je geopandas???
import json
from geopy.exc import GeocoderTimedOut

##### Widgets ############
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
import os
from IPython.display import Image

##### Bonus - Hudba ############
import winsound

##### Vizualizace ############
import seaborn as sns



import sys

-----------------------
# 2. Definice všech funkcí
-----------------------

## 2a) Dílčí funkce

In [387]:

################################################################################################################################
##############################################   SCRAPING    ###################################################################
################################################################################################################################

################################################################################################################################  1 = Scraping

def get_soup_elements(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):  
    
    browser = webdriver.Chrome()
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby

    browser.get(url) 
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ??
    
    elements = []    
    
    for link in soup.findAll('a', attrs={'href': re.compile("^/detail/")}):     
        link = link.get('href')   
        elements.append(link)     
    elements = elements[0::2]   

    ##########################################
    # 3. zjištění počtu listů - mělo by být optional, ale nevadí
    ##########################################
    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    ##########################################
    # 4. nastavení počtu stránek  -mělo být víc promakané
    ##########################################
    if pages == 999:      
        pages = max_page
    
    print("Scrapuji (pouze) " + str(pages) + " stran.")
    print("----------------")
    
    ##########################################
    # 4. Scrapping zbylých listů - naštěstí v jednom okně
    ##########################################    
    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml') 
        
        elements2 = []
        
        for link in soup2.findAll('a', attrs={'href': re.compile("^/detail/prodej/")}):  
            link = link.get('href') 
            elements2.append(link)  
   
        elements2 = elements2[0::2]  
        
        elements = elements + elements2
    
    browser.quit()   
    
    return elements

################################################################################################################################  2 = Získání URLS
def elements_and_ids(x):
    
    elements = pd.DataFrame({"url":x})

    def get_id(x):
        x = x.split("/")[-1]
        return x

    elements["url_id"] = elements["url"].apply(get_id)
    
    len1 = len(elements)
    elements = elements.drop_duplicates(subset = [ "url", "url_id"], keep = "first", inplace = False)   #Přidáno nově, v tuto chvíli odmažu duplikáty a jsem v pohodě a šetřím si čas dál.
    len2 = len(elements)                                                                               ## BTW asi to enní chyba těch, co zadávají inzeráty, ale scrapingu, on stáhne třeba pět adres co jsou za sebou dvakrát,
                                                                                                      # možná protože se mezitím změnily inzeráty nebo stránka nebo něco.. koeneckonců stahuje 30-50 min :/
    print("-- Vymazáno " + str(len2-len1) + " záznamů kvůli duplikaci.")
    return elements

################################################################################################################################  3 = získání Souřadnic, Ceny a Popisu = z JSON
def get_coords_price_meters(x):
    
    url = "https://www.sreality.cz/api/cs/v2/estates/" + str(x)
    
    response = requests.get(url)
    byt = json.loads(response.text, encoding = 'UTF-8')                                
    try:
        coords = (byt["map"]["lat"], byt["map"]["lon"])
    except:
        coords = (0.01, 0.01)
    try:
        price = byt["price_czk"]["value_raw"] 
    except:
        price = -1
        
    try:
        description = byt["meta_description"]
    except:
        description = -1
    
    return coords, price, description


################################################################################################################################  4 = Prodej + Dům + Pokoje = z URL
def characteristics(x):
    x = x.split("/")
    buy_rent = x[2]
    home_house = x[3]
    rooms = x[4]
    
    return buy_rent, home_house, rooms

################################################################################################################################  5 =  Plocha z Popisu- VYLEPŠENO

# Upravenopro čísla větší než 1000 aby je to vzalo
# Zároveň to už nebere Dispozice, Atpyický atd.

def plocha(x):
    try:
        metry = re.search(r'\s[12]\s\d{3}\s[m]', x)[0] # SPecificky popsáno: Začíná to mezerou, pak 1 nebo 2, pak mezera, pak tři čísla, mezera a pak "m"
        metry = metry.split()[0] + metry.split()[1]     # Separuju Jedničku + stovky metrů, bez "m"
    except:
        try:
            metry = re.search(r'\s\d{2,3}\s[m]', x)[0]  #Mezera, pak 1-3 čísla, mezera a metr
            metry = metry.split()[0]                    # Separuju čísla, bez "m"
        except:
            metry = -1
    return metry

data["plocha"] = data['popis'].apply(plocha)

################################################################################################################################  6 = Adresy z předešlých inzerátů a short_coords

def short_coords(x):
    """
    x = x.astype(str)   # Bylo potřeba udělat string - ale Tuple se blbě převádí - vyřešil jsem uložením a načtením skrz excel
    """
    
    x1 = re.split(r'\W+', x)[1] + "."+re.split(r'\W+', x)[2]
    x1 = round(float(x1), 4)

    x2 = re.split(r'\W+', x)[3] + "."+re.split(r'\W+', x)[4]
    x2 = round(float(x2), 4)

    return (x1, x2)

#############################

def adress_old(x):  # Napmapuje až 80 % Adres z předešlých inzerátů

    adresy = pd.read_excel("Adresy.xlsx")
    adresy = adresy[["oblast", "město", "okres", "kraj", "url_id", "short_coords"]]   # Odebral jsem Počet _celkem, udělám ho separátně potom
    
    #Nejlepší napárování je toto:
    # porovnáno Inner a Left minus řádky s NaNs a funguje stejně)
    
    x.short_coords = x.short_coords.astype(str)   # Nově asi musím přidat toto - získat string na souřadnice, protože v Načteném adresáři je mám už taky jako string
    data = pd.merge(x, adresy, on=["short_coords", "url_id"], how = "left")  #upraveno matchování na url_ID + short_coords, je to tak iv Adresáří, je to jednoznačné, jsou tam unikátní. 
                                                                            # Pokud si v dalším kroku dostáhnu ke starému url_id a k nové coords ještě novou adresu, tak pak se mi uloží do Adresáře nová kombiance ID + short_coord a je to OK
                                                                             # Viz funkce"update_databáze_adres() kde je totéž info
            
    print("--Počet doplněných řádků je: " + str(len(data[~data.kraj.isna()])) + ", počet chybějících řádků je: "   + str(len(data[data.kraj.isna()])))
    
    return data

################################################################################################################################ 7 = Adresy - zbývající přes GeoLocator

def adress_new(x):

# Pozn. - je to random, závislost rychlosti na user_agent, i na format_string se nepovedlo potvrdit - ale dokumentace user-agent uvíádí jako povinnost
# Naopak timeout na 20s a na None a zpátky mi nepomohl s rychlostí, ale minimálně zrušil Errory - None záleží na verzi geopy !! viz dokumentace
# Nejvíc podezřívám Wifi vs. kabel - 3 sec -> 1/3 sec, skoro vždy -> Ano, kabel v pokoji = 3-20s na záznam, kabel v obýváku = 0.25s na záznam !!!!!!!!!!!!!!!!§
# Problém s Too many requests se "spraví přes noc", kdyžtak - nebo viz stackoverflow - nastavit user-agent (https://stackoverflow.com/questions/22786068/how-to-avoid-http-error-429-too-many-requests-python)
  
    geolocator = Nominatim(timeout = 20, user_agent = "specify_your_app_name_here")           
    location = geolocator.reverse(x.strip("())"))   ### Nevím proč, ale najednou to po mně nechce STRIP, jen X samotné, pak zas měním zpátky
                                                    # Reverse samotné znamená obrácené vyhleádvání = souřadnice -> Adresa
     
    try:
        oblast  = location[0].split(",")[-7]
    except:
        oblast  = -1
    try:
        město = location[0].split(",")[-6]
    except:
        město  = -1
    try:
        okres = location[0].split(",")[-5]
    except:
        okres  = -1
    try:
        kraj = location[0].split(",")[-4]  
    except:
        kraj  = -1       
    
    #time.sleep(0.5)
    return oblast, město, okres, kraj

##################################################################

# Není nutné, půjde i bez toho - když vypustím TIMEOUT - zatím ale nechávám
def repeat_adress(x):
    try:
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))
    except GeocoderTimedOut:
        print("Another try")
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))


################################################################################################################################ 8 = Merging adres

def adress_merging(x):

    data_new = x.copy()          
    bool_series = pd.isnull(data_new.kraj)                                   
    data_new = data_new[bool_series]     #subset s chybějícími adresami, už musím mít hotové ADRESS_OLD !!!!     
        
    # Musím takhle přímo použít na subset, data_new se tím updatuje
    # Ideální případ: cca 3561/3 = 1.200s = 20 min bez záseku -> vše chybějící
    repeat_adress(data_new)
        
    # Nakonec jsem zamítl .merge( all columns, how = "outer"), kdoví proč to dávalo 7 řádků navíc a tohle je čistší
    data_all = pd.concat([x, data_new], join_axes=[x.columns])   
    data_all = data_all[~pd.isnull(data_all.kraj)]
    data_all = data_all.sort_index()

    data = data_all.copy()
    
    return data

################################################################################################################################
##############################################   CLEANING    ###################################################################
################################################################################################################################

################################################################################################################################ 1 = Mezery u Adres

############ Mezery před názvy (všeho) a pomlčky doplněné z GeoPy
def smaž_mezery(x):
    x = re.findall(r'\w+', str(x))   # Když napíšu tento regex (r'\w+-?\w+'), tak bych i zachoval "pomlčku", 
    x = ' '.join(x)                  # ale to já naopak nechci, vyřeším si dvě věci najednou 
    
    return x

################################################################################################################################ 2 = Posunutý Okres, Kraj...

def kraj_check(x):
    x = x.split()[0]
    return x


def uprav_adresy(x):

    x["kraj_check"] = x["kraj"].apply(kraj_check)

    # U těch řádků, kde byl okres v názvu kraje posuň všechny tři hodnoty "doleva"
    x.oblast[x["kraj_check"] == "okres"] = x.město[x["kraj_check"] == "okres"].copy()
    x.město[x["kraj_check"] == "okres"] = x.okres[x["kraj_check"] == "okres"].copy()
    x.okres[x["kraj_check"] == "okres"] = x.kraj[x["kraj_check"] == "okres"].copy()
    x.kraj[x["kraj_check"] == "okres"] = -1

    # Oříznutý dataset OBCE
    obce = pd.read_excel(r"obce_okresy_kraje.xlsx")
    obce_kus = obce[["okres", "kraj"]].copy()

    # LEFT Merge by měl přiřadit okresům správné Kraje z df obcí
    data = pd.merge(x, obce_kus, on=['okres'], how = "left")

    # Smazání případných duplikátů
    data.drop_duplicates(keep = "first", inplace = True)

    # Přiřazení nového sloupečku na starý
    data.kraj_x[data["kraj_check"] == "okres"] = data.kraj_y[data["kraj_check"] == "okres"].copy()

    # Přejmenování a získání lepších sloupců
    data = data.rename(columns={'kraj_x': 'kraj'})
    data = data[["popis", "prodej", "dům", "pokoje", "plocha", "oblast", "město", "okres", "kraj", "cena", "url", "url_id", "coords", "short_coords"]]

    data.reset_index(inplace = True, drop = True)

    return data
################################################################################################################################ 3 = Počet obyvatel z Excelu ("obce_okresy_kraje.xlsx")

def počet_obyvatel(x):

    obce = pd.read_excel("obce_okresy_kraje.xlsx")
    obce_kus = obce[["město", "okres", "kraj", "obyvatelé"]].copy()
    data = pd.merge(x, obce_kus ,on = ["město", "okres", "kraj"], how = "left")
    data.drop_duplicates(keep = "first", inplace = True)

    return data

################################################################################################################################ 4 = Update databáze adres

def update_databáze_adres(x):
    
    adresy = pd.read_excel("Adresy.xlsx")
    len1 = len(adresy)
    print("Původní délka adresáře: " + str(len1))

    #Oříznu si data na to, co chci uložit a přidám aktuální datum
    data2 = x[["oblast", "město", "okres", "kraj", "url", "url_id", "short_coords"]]  # Odebráno Coords, vznikaly duplikáty zbytečně
    data2["Datum"] = datetime.datetime.now().strftime("%Y-%m-%d")

    # Připojím nová data, dropnu duplikáty (mažu odzadu), resetnu indexy, změřím délku
    # Přírůstek by měl odpovídat počtu řádků dohledaných přes Geopy minus duplikáty případně
    adresy = pd.concat([adresy, data2])
    adresy = adresy.drop_duplicates(subset = [ "url_id", "short_coords"], keep = "first", inplace = False)
    
    # Změněna logika, dropni všechno kde máš dvojí varianty pro kombinaci url_id + short_coords, protože přesně tohohle se to mapuje a dělá to pak rozdvojení různá. !!
    # Jinak ponechává to možnost že mám jedno ID vícekrát s různými short_coords, ale to je možné, nebrání to zpětnému mapování
    adresy.reset_index(inplace = True, drop = True)
    len2 = len(adresy)
    print( "Nová délka adresáře: " + str(len2) + ", přidáno "+ str(len2-len1) + " záznamů. Ukládám do Adresy.xlsx")

    return adresy


################################################################################################################################ 5 = Cena na tisíce

def cena_tisíce(y):

    y["cena"] = y["cena"].apply(lambda x: int(round(x/1000, 0)))

    return y

################################################################################################################################ 6 = Cena za metr

def cena_metr(y):

    y["cena_metr"] = y["cena"].astype(int)/y["plocha"].astype(int)
    y["cena_metr"] = y["cena_metr"].apply(lambda x: round(x, 1))
    
    return y

## 2b) Tři hlavní funkce - SCRAPING, CLEANING, DROPING

In [386]:

#Volba mezi tím nechat to defaultní, nebo si to předem vždycky nadefinovat.. ?
#Nadefinovat, abych mohl správně pojmenovat Excely, i když zatím jedu jen jednu kategorii pořád... !!!!!!
# !!! Přidat informace o počtu řádků, průběžně- např u Adres - kolik bylo, kolik doplním. Dále u Mazání a Duplikátů


################################################################################################################################
##############################################   SCRAPING    ###################################################################
################################################################################################################################

def scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):
    
    # Scrapni data - hezky komunikuje = cca 50 min
    data = get_soup_elements(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, pages = pages)
    print( "1/8 Data scrapnuta, získávám URLs.")
    
    # 2 = Získání URLS
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...")
    
    
    # 3 = získání Souřadnic, Ceny a Popisu = z JSON
    data["coords"], data["cena"], data["popis"] = zip(*data['url_id'].map(get_coords_price_meters))
    data.to_excel(r"a2_Souřadnice_prodej_byty.xlsx")
    print( "3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.")
   
    # 4 = Prodej + Dům + Pokoje = z URL
    data["prodej"], data["dům"],  data["pokoje"] = zip(*data['url'].map(characteristics))
    print("4/8 Získány informace z URLs, nyní získávám informace z popisu.")
    
    # 5 =  Plocha z Popisu
    data["plocha"] = data['popis'].apply(plocha)
    data.to_excel(r"a3_Popisky_prodej_byty.xlsx")
    print( "5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.")
    
   
    # 6 = Adresy z předešlých inzerátů a short_coords
    data = pd.read_excel(r"a3_Popisky_prodej_byty.xlsx")   # Abych se vyhnul konverzi TUPLE na STRING, což není triviální, tak si to radši uložím a znova načtu a získám stringy rovnou. Snad mi to nerozbije zbytek
    data["short_coords"] = data["coords"].apply(short_coords)
    data_upd = adress_old(data)                                # Tady nepotřebuji maping, protože se nesnažím něco nahodit na všechny řádky, ale merguju celé datasety
    data = data_upd.copy()
    print( "6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...")            # Přidat do printu počet řádků, kolik mám a kolik zbývá v 7. kroku

    # 7-8 = Adresy - zbývající přes GeoLocator + Merging
    data_upd = adress_merging(data)
    data = data_upd.copy()
    data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
    print("7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    

    return data

In [385]:

################################################################################################################################
##############################################   CLEANING    ###################################################################
################################################################################################################################

def clean_all(x):
    
    #data = pd.read_excel(x)   = Pracovní verze s načítáním dat, jinak napojeno na Scrap
    data = x.copy()
    
    print("----------------")
    print("Počet záznamů před čištěním: " + str(len(data)))
    
    
    # 1 = Mezery u adres 
    data["oblast"] = data["oblast"].apply(smaž_mezery)
    data["město"] = data["město"].apply(smaž_mezery)
    data["okres"] = data["okres"].apply(smaž_mezery)
    data["kraj"] = data["kraj"].apply(smaž_mezery)
    print("1/6 Vymazány mezery před názvy, následuje posouvání rozbitých okresů a krajů")
    
    # 2 = Posunutý Okres, Kraj...
    data = uprav_adresy(data)
    print("2/6 Posunuty rozbité okresy a kraje, následuje doplnění počtu obyvatel.")
    
    # 3 = Počet obyvatel z Excelu
    data = počet_obyvatel(data)
    data.to_excel("a5_Adresy_a_obyvatele_data_prodej_byty.xlsx")
    print("Aktuální délka datasetu: " + str(len(data)))
    print("3/6 Doplněn počet obyvatel, updatuji databázi adres.")
    
    # 4 = Update databáze adres 
    adresy = update_databáze_adres(data)
    adresy.to_excel(r"Adresy.xlsx")
    print("4/6 Databáze Adres updatována. Délka datasetu: " + str(len(data)) + " záznamů. Převádím ceny na tisíce")    

    # 5 Cena na tisíce
    data = cena_tisíce(data)
    print("5/6 Ceny převedeny na tisíce. Počítám ceny za metr.")
    
    # 6 Cena za metr
    data = cena_metr(data)
    data = data[["popis", "prodej", "dům", "pokoje", "plocha", "oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "url", "url_id", "coords", "short_coords"]]
    data.to_excel("a6_CLEANED_data_prodej_byty.xlsx")
    print("6/6 Spočítány ceny za metr. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 2.")
    
          
    return data


In [ ]:
## NA POZDĚJI

def dropping(data):
    
    return data

-----------------------
# 3. Spuštění
-----------------------

In [35]:
scrap_all.isin([-1]).any()

url             False
url_id          False
coords          False
cena             True
popis           False
prodej          False
dům             False
pokoje          False
plocha          False
short_coords    False
oblast          False
město           False
okres           False
kraj            False
dtype: bool

In [ ]:

scrap_all = scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 4)
scrap_all.head()


In [381]:

data = clean_all("a_Adresy_prodej_byty_na fázi 2.xlsx")
#data = clean_all(scrap_all)
data.head()


----------------
Počet záznamů před čištěním: 80
1/6 Vymazány mezery před názvy, následuje posouvání rozbitých okresů a krajů


C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:275: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3

2/6 Posunuty rozbité okresy a kraje, následuje doplnění počtu obyvatel.
Aktuální délka datasetu: 80
3/6 Doplněn počet obyvatel, updatuji databázi adres.
Původní délka adresáře: 14844
Nová délka adresáře: 14844, přidáno 0 záznamů. Ukládám do Adresy.xlsx


C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4/6 Databáze Adres updatována. Délka datasetu: 80 záznamů. Převádím ceny na tisíce
5/6 Ceny převedeny na tisíce. Počítám ceny za metr.
6/6 Spočítány ceny za metr. Celková délka datasetu: 80. Konec Fáze 2.


,popis,prodej,dům,pokoje,plocha,oblast,město,okres,kraj,obyvatelé,cena,cena_metr,url,url_id,coords,short_coords
0,"Byt 4+kk 133 m² k prodeji Smečenská, Slaný; 4 ...",prodej,byt,4+kk,133,Smečenská,Slaný,okres Kladno,Středočeský kraj,15834,4400,33.1,/detail/prodej/byt/4+kk/slany-slany-smecenska/...,340774492,"(50.227294430378, 14.0833352740144)","(50.2273, 14.0833)"
1,"Byt 4+kk 77 m² k prodeji Smečenská, Slaný; 3 8...",prodej,byt,4+kk,77,Smečenská,Slaný,okres Kladno,Středočeský kraj,15834,3890,50.5,/detail/prodej/byt/4+kk/slany-slany-smecenska/...,3487092316,"(50.227294430378, 14.0833352740144)","(50.2273, 14.0833)"
2,"Byt 3+kk 87 m² k prodeji Višňová, Moravany; 6 ...",prodej,byt,3+kk,87,Moravany u Brna,Moravany,okres Brno venkov,Jihomoravský kraj,3070,6800,78.2,/detail/prodej/byt/3+kk/moravany-moravany-visn...,674836060,"(49.1533615129803, 16.590165452052)","(49.1534, 16.5902)"
3,"Byt 3+kk 115 m² k prodeji Vršovická, Praha 10 ...",prodej,byt,3+kk,115,Vršovice,Praha,okres Hlavní město Praha,Hlavní město Praha,1308632,10500,91.3,/detail/prodej/byt/3+kk/praha-vrsovice-vrsovic...,1748577884,"(50.065844, 14.451906)","(50.0658, 14.4519)"
4,"Byt 3+1 80 m² k prodeji Třebízského náměstí, P...",prodej,byt,3+1,80,Třebízského náměstí,Postoloprty,okres Louny,Ústecký kraj,4716,1749,21.9,/detail/prodej/byt/3+1/postoloprty-postoloprty...,1480142428,"(50.3619343716, 13.6956684270551)","(50.3619, 13.6957)"
5,"Byt 2+kk 41 m² k prodeji Pražská, Teplice; 690...",prodej,byt,2+kk,41,Pražská,Teplice,okres Teplice,Ústecký kraj,49575,690,16.8,/detail/prodej/byt/2+kk/teplice-teplice-prazsk...,1211706972,"(50.6369557751961, 13.8399243348652)","(50.637, 13.8399)"
6,"Byt 3+1 75 m² k prodeji Zimní, Jihlava - Horní...",prodej,byt,3+1,75,Horní Kosov,Jihlava,okres Jihlava,Kraj Vysočina,50845,2850,38.0,/detail/prodej/byt/3+1/jihlava-horni-kosov-zim...,2480094812,"(49.4050713405544, 15.5609464898876)","(49.4051, 15.5609)"
7,"Byt 1+kk 35 m² k prodeji Kojetická, Neratovice...",prodej,byt,1+kk,35,Kojetická,Neratovice,okres Mělník,Středočeský kraj,16163,2190,62.6,/detail/prodej/byt/1+kk/neratovice-neratovice-...,3311156828,"(50.2528973169751, 14.5196175798477)","(50.2529, 14.5196)"
8,"Byt 3+1 64 m² k prodeji Kosmonautů, Pardubice ...",prodej,byt,3+1,64,Polabiny II,Pardubice,okres Pardubice,Pardubický kraj,90688,2999,46.9,/detail/prodej/byt/3+1/pardubice-polabiny-kosm...,3032694364,"(50.0514721733985, 15.7597388697676)","(50.0515, 15.7597)"
9,"Byt 1+1 33 m² k prodeji Štouračova, Brno - Bys...",prodej,byt,1+1,33,Bystrc,Brno,okres Brno město,Jihomoravský kraj,380681,2450,74.2,/detail/prodej/byt/1+1/brno-bystrc-stouracova/...,3896061532,"(49.2158416955, 16.5186408895)","(49.2158, 16.5186)"


In [388]:
#&&&&
#### SCRAP + CLEAN Dohromady:

pokus = clean_all(scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 4))
pokus.head()

----------------
Scrapuji: prodej byty
Celkem inzerátů: 13905
Celkem stránek: 696
Scrapuji (pouze) 4 stran.
----------------
Strana 3 = 75.0% progress. Zbývá cca: 3.4 sekund.1/8 Data scrapnuta, získávám URLs.
-- Vymazáno 0 záznamů kvůli duplikaci.
2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...
3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.
4/8 Získány informace z URLs, nyní získávám informace z popisu.
5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.
--Počet doplněných řádků je: 55, počet chybějících řádků je: 25
6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...
7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: 80. Konec Fáze 1.
----------------
Počet záznamů před čištěním: 80
1/6 Vymazány mezery před názvy, následuje posouvání rozbitých okresů a krajů


C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:275: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3

2/6 Posunuty rozbité okresy a kraje, následuje doplnění počtu obyvatel.
Aktuální délka datasetu: 80
3/6 Doplněn počet obyvatel, updatuji databázi adres.
Původní délka adresáře: 14844
Nová délka adresáře: 14869, přidáno 25 záznamů. Ukládám do Adresy.xlsx


C:\Users\jiri.zelenka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4/6 Databáze Adres updatována. Délka datasetu: 80 záznamů. Převádím ceny na tisíce
5/6 Ceny převedeny na tisíce. Počítám ceny za metr.
6/6 Spočítány ceny za metr. Celková délka datasetu: 80. Konec Fáze 2.


,popis,prodej,dům,pokoje,plocha,oblast,město,okres,kraj,obyvatelé,cena,cena_metr,url,url_id,coords,short_coords
0,"Byt 3+1 82 m² k prodeji Stehlíkova, Praha 6 - ...",prodej,byt,3+1,82,Suchdol,Praha,okres Hlavní město Praha,Hlavní město Praha,1308632.0,5990,73.0,/detail/prodej/byt/3+1/praha-suchdol-stehlikov...,722005596,"(50.1347808252759, 14.3767857485312)","(50.1348, 14.3768)"
1,"Byt 1+1 43 m² k prodeji Štefánikova, Praha 5 -...",prodej,byt,1+1,43,Smíchov,Praha,okres Hlavní město Praha,Hlavní město Praha,1308632.0,4600,107.0,/detail/prodej/byt/1+1/praha-smichov-stefaniko...,209862236,"(50.0754402778, 14.4042047222)","(50.0754, 14.4042)"
2,"Byt 2+1 54 m² k prodeji Jožky Jabůrkové, Ústí ...",prodej,byt,2+1,54,Bukov,Ústí nad Labem,okres Ústí nad Labem,Ústecký kraj,92952.0,1770,32.8,/detail/prodej/byt/2+1/usti-nad-labem-bukov-jo...,1634025052,"(50.6830432997364, 13.9997331354459)","(50.683, 13.9997)"
3,"Byt 2+1 53 m² k prodeji Jožky Jabůrkové, Ústí ...",prodej,byt,2+1,53,Bukov,Ústí nad Labem,okres Ústí nad Labem,Ústecký kraj,92952.0,1770,33.4,/detail/prodej/byt/2+1/usti-nad-labem-bukov-jo...,1536573020,"(50.6827879704248, 14.0003759561062)","(50.6828, 14.0004)"
4,"Byt 2+kk 39 m² k prodeji Bochovská, Praha 5 - ...",prodej,byt,2+kk,39,Jinonice,Praha,okres Hlavní město Praha,Hlavní město Praha,1308632.0,3990,102.3,/detail/prodej/byt/2+kk/praha-jinonice-bochovs...,3668328028,"(50.052837912814, 14.3657943222549)","(50.0528, 14.3658)"


In [380]:
data = pd.read_excel("a_Adresy_prodej_byty_na fázi 2.xlsx")
data.groupby("plocha")["url_id"].count()

plocha
32     1
33     1
34     1
35     1
37     1
39     1
40     1
41     1
43     2
44     3
45     1
46     1
50     3
52     2
53     1
54     2
56     2
58     3
59     1
61     1
62     3
63     2
64     3
65     2
66     4
69     2
70     2
71     1
72     1
73     2
75     1
77     1
79     2
80     3
81     1
84     2
85     1
87     1
89     1
90     1
93     1
96     1
101    1
104    1
107    1
109    1
115    1
118    1
119    2
126    1
133    1
137    1
158    1
Name: url_id, dtype: int64

## 4e) Dropnutí Bez Cen // ((Plochy)) // Pokojů // Města // Kraje // Okresu

* Asi zvlášť problémy s geografií, a plochou pak celaring !!!

*  ze 12.498:
* CENA : 897 je bez ceny (k dostání u RK) - Smažu
* PLOCHA: 503 je bez plochy - nemazal bych, není nutná
* POKOJE - ty jsou všude :)
* ADRESY  55 nemá ani kraj, okres, nic (chybí coordinates, i když by šlo z popisku) - Smažu
* Bez ceny a be z Coordinates se celkem překrývá, celkem 908

* CELKEM 12,498 -> ?? kompletních záznamů

In [ ]:
print("Celkem záznamů bez ceny: "+ str( len(data[data["cena"] == 0]) )   )
print("Celkem záznamů bez plochy: "+ str( len(data[data["plocha"] == -1]) )  +". Nemažu je." )
print("Celkem záznamů bez pokojů: "+ str( len(data[data["pokoje"] == -1]) )   )
print("Celkem záznamů bez oblasti a okresu: "+ str(len(data[data["coords"] == "(0.01, 0.01)"])))

x = len(data)

def clearing(data):

    data = data[data["cena"] != 0]   # Pozn - bylo -1, ale pak se dělilo tisícem a zaokrouhlovalo.
    display(len(data))
    #data = data[data["plocha"] != -1]
    #display(len(data))
    data = data[data["coords"] != "(0.01, 0.01)"]
    display(len(data))
   
    
    
    print("Finální délka datasetu: " + str(len(data)) + ", smazáno " + str(x-len(data)) + " záznamů.")
    
    return data
###############################################################

data = clearing(data)
data

## 4f) Dropnutí extrémů = pochybné záznamy, MANUÁLNĚ

In [ ]:
data = data[data["cena"] != 12]

x = data.index[data["cena"] == 320].tolist()  # Je jen jeden, naštěstí, jinak bych musel líp filtrovat
data  = data[data.index != x]

x = data.index[(data["cena"] == 180) & (data["město"] == "Praha")].tolist()  # Upřesněno na pražský byt
data  = data[data.index != x]
len(data)

## 4g) Pokus o hledání duplikátů


Shodnost ve všem kromě popisu a coords - 494 záznamů (2.239 bez shodnosti ceny)
        #3658	URL_ID: 1956396636	(50.071137083957, 14.4703357132463)	 - Míří na střed ulice
        #7096	URL_ID: 4064231004	(50.0724406380103, 14.4713951723744) - Míří na dům přímo	

Shodnost ve všem kromě Coords - 211 

Shodnost ve všem kromě popisu - 309
        #134   4096974428   - Popis Neratovice
        #140   1482702428   - Popis Předboj

Shodnost ve všem včetně včetně popisu - 189  
#####################################################################


Rozdíl Komplet identických včetně Ceny  bez ceny se liší o 19 záznamů, asi tedy smažu jen ty komplet stejné, včetně ceny, to by měla být tutovka !!

Druhý krok - koukám že popisy jsou někdy fakt jen drobnost, takže mažu i podle nich

Třetí krok - zbylo mi 20x2 záznamů, pozorování: Všechny dvojice mají téměř identické souřadnice - mažu taky !!

Otázka - dají se tři kroky nahradit jedním, tj smazat vše kde se neshoduje popisek ani cena (nebo jen jedno z toho)
to asi nevyjde stejně, protože to zabere i ty co se rozchází ve dvou věcech, a ty můžou být už opravdu jiné..?

In [ ]:

data_duplic = data[data.duplicated([  "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "cena", "coords"  ],keep= "first")]
#Mělo by být jedno, jestli definující sloupce dám dovnitř duplicated, nebo za druhé DATA, ale pak musím přidat[] navíc
# False -> Ponech si všechny zduplikované, "first" -> ponech si první výskyt

display(len(data_duplic))

display(data_duplic.sort_values("plocha").head(15))
# ZObrazí duplikované výsledky - pokud nahoře zvolím False, tak všechny výsledky se svými dvojníky

seznam = list(data_duplic.index)
data = data[ ~data.index.isin(seznam)]
display(len(data))

# !! Tímto ale smažu jen první výskyty z dat !

###############################################################################################################################################x
# Podruhé pro druhou skupinu shodností, nechci oboje spojit dohromady, protože bych povolil dvojitou chybu
# řekl bych že to je dobře. sice mi zbývá asi 150 potenciálních duplikátů, ale hned namátkově jsou ta mdva různé byty. Spíš to vypadá že se tam pricing řídí lokalitou a plochou, proto to má smysl chatne oba byty

data_duplic = data[data.duplicated([ "popis",  "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "cena"  ],keep= "first")]
display(len(data_duplic))

display(data_duplic.sort_values("plocha").head(15))

seznam = list(data_duplic.index)
data = data[ ~data.index.isin(seznam)]
display(len(data))

# 11.255 zbývá z 11.585 = 330 smazáno

## POznámka: data.iloc[1101] = 1101. záznam podle klasického řádkování
##           data.ix[1101] = prvek s idnexem 1101. Pokud ten nebyl resetován tak se to liší

-----------------------
# 5. Analýzy
* https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
-----------------------


#### 0) Nadefinované grafické pomůcky
----
#### 1) Počty inzerátů
 * 1.a) Podle místností
 * 1.b) Podle měst/okresu/kraje
---
#### 2) Ceny
- 2.a) Podle místností
- 2.b) Podle měst
- - 2.b.1) Nejdražší města
- - 2.b.2) Nejlevnější města
- 2.c) Podle okresů
- - 2.c.1) Nejdražší okresy
- - 2.c.2) Nejlevnější okresy
- 2.d) Podle krajů
-----------------------
#### 3) Ceny podle místností a oblastí PRAHA & MOST
- 3.a) Ceny podle místností Praha
- 3.b) Ceny městských částí Prahy
- - 3.b.1) Nejdražší městské části
- - 3.b.2) Nejlevnější pražské části
- 3.c) Ceny podle místností Most
- 3.d) PRAHA vs MOST
-----------------------
#### 4) Pivot - Pražské části a pokoje
-----------------------
#### 5) Analýzy podle okresů a krajů = DODĚLAT
----
#### 6) Interaktivní widgety = DODĚLAT
---

### 0) Nadefinované grafické pomůcky

In [ ]:
### Barvičky do BARů

# červená a zelená jsou vidět na html exportu, modrá a rudá s černým písmem NE
# Takže na HTML export musím místo modré dát tyrkys...

zelená = "#22c02c"
tyrkys =   "#16c9ea"
modrá  =    "#0307ca"
červená =  "#e61717"
rudá =      "#a10000"
hnědá = "#4d0000"


# 1.) Zkusi barevnou škálu na celou tabulku. tím by se našly extrémy co nejdou podle tmavosti
# 2) Pivot - udělat jednak potlačení nul, a pak barevnou paletu?
# 3) Zkusit nějak aplikovat ten sloupec s dvěma arvami, ale musím mít kladné i záporné hodnoty, nějaké odchylky od průměru - to bych asi msuel třeba na všechny okresy najednou nebo něco


########################################################################

def small_red(val):
 
    color = 'red' if val < 500 else 'white'
    return 'color: %s' % color

 #.style.applymap(small_red)
########################################################################

def highlight_max(s):
 
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

#  .style.apply(highlight_max) 

########################################################################
def highlight_median(s):   # Vyznačí nejvyšší hodnotu ve sloupci

    is_max = s == s.median()
    return ['background-color: green' if v else '' for v in is_max]

#  .style.apply(highlight_median) 


########################################################################
########################################################################

# https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html

"""
from matplotlib.colors import LinearSegmentedColormap

def grayscale_cmap(cmap):
    Return a grayscale version of the given colormap
    cmap = plt.cm.get_cmap(cmap)
    colors = cmap(np.arange(cmap.N))
    
    # convert RGBA to perceived grayscale luminance
    # cf. http://alienryderflex.com/hsp.html
    RGB_weight = [0.299, 0.587, 0.114]
    luminance = np.sqrt(np.dot(colors[:, :3] ** 2, RGB_weight))
    colors[:, :3] = luminance[:, np.newaxis]
        
    return LinearSegmentedColormap.from_list(cmap.name + "_gray", colors, cmap.N)
    

def view_colormap(cmap):
    Plot a colormap with its grayscale equivalent
    cmap = plt.cm.get_cmap(cmap)
    colors = cmap(np.arange(cmap.N))
    
    cmap = grayscale_cmap(cmap)
    grayscale = cmap(np.arange(cmap.N))
    
    fig, ax = plt.subplots(2, figsize=(6, 2),
                           subplot_kw=dict(xticks=[], yticks=[]))
    ax[0].imshow([colors], extent=[0, 10, 0, 1])
    ax[1].imshow([grayscale], extent=[0, 10, 0, 1])

view_colormap('viridis')
"""
#

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Ukaž/Schovej všechen kód"></form>''')

###  LOAD (or save)

In [ ]:
data = pd.read_excel(r"data_prodej_byty.xlsx")
print("Data mají " + str(len(data)) + " řádků." )

data_old = pd.read_excel(r"data_prodej_byty_OLD.xlsx")
print("Stará data mají " + str(len(data_old)) + " řádků. - Před pročištěním." )

obce = pd.read_excel("obce_okresy_kraje.xlsx")
print("Obce mají " + str(len(obce)) + " řádků." )


display(data.head())
display(data_old.head())
display(obce.head())

In [ ]:
data.info()

### 1) Počty inzerátů

### 1.a) Počty inzerátů podle místností

In [ ]:
# data.groupby('pokoje')['popis'].count().to_frame()  #.sum()

#data.pokoje.value_counts()
#data.pokoje.value_counts().to_frame()
data.pokoje.value_counts().to_frame().style.bar(align = "mid", color = tyrkys)


### 1.b) Počty inzerátů podle měst/okresu/kraje

In [ ]:
########################################################
# Třikrát identické funkce !! ale "nunique" hází jiný výsledek než "count" a "size" při kombinaci s "nlargest" !!!
########################################################
data.groupby('město')['popis'].nunique()
data.groupby('město')['popis'].count()   # COunt by mělo počítat všechny, nejen UNIQUE !!
data.groupby('město')['popis'].size()
########################################################

In [ ]:
# Všechna
data.groupby(['město', "okres"])['popis'].count().nlargest(10).to_frame().style.bar(align = "mid", color = modrá)

In [ ]:
# Bez Prahy
data[data["město"] != "Praha"].groupby(['město', "okres"])['popis'].count().nlargest(10).to_frame().style.bar(align = "mid", color = modrá)

# Nový kod jde připojit i za tohle :3
# Upraveno align na mid, aby ůstaly zachované délky sloupců

In [ ]:
# Všechny
data.groupby('okres')['popis'].count().nlargest(10).to_frame().style.bar(align = "mid", color = modrá)

In [ ]:
# Bez Prahy
data[data["město"] != "Praha"].groupby( "okres")['popis'].count().nlargest(10).to_frame().style.bar(align = "mid", color = modrá)

In [ ]:
data.groupby('kraj')['popis'].count().nlargest(20).to_frame().style.bar(align = "mid", color = modrá)

### 2) Ceny

### 2.a) Ceny podle místností

In [ ]:
data["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).to_frame()


In [ ]:
data.groupby("pokoje")["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)
#data[data["plocha"] != -1].groupby("pokoje")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

### extrém 12.000 !!! - vyškrtnuto

#data[data["cena_metr"] == 568.3]
#data[data["cena_metr"] == 1]


### 2.b) Ceny podle měst

In [ ]:
data["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x))

In [ ]:
data[data["plocha"] != -1]["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x))


### 2.b.1) Nejdražší města

In [ ]:
data.groupby(["město", "okres"])["cena"].agg(["min",  np.median, "mean","max", "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)

data[data["plocha"] != -1].groupby(["město", "okres"])["cena_metr"].agg(["min",  np.median, "mean","max", "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)

# Subset jen říká, kam chci barvu

# Upravený kod, jde řadit nLargest i podle jednoho z více prvků !!
# upravený kod, jde formátovat i některé sloupce
# 
# V1 :nechám agg( [sloupec ]) a pak musím definovat který chci v Nlargest ale mám frame a nemusím dávat "to_frame()"
# V2: nechám jen agg(sloupec) a pak nemusím definovat podle kterého chci řadit ale zas musím udělat to_frame()

### 2.b.2) Nejlevnější města

In [ ]:
data.groupby(["město", "okres"])["cena"].agg(["min",  np.median, "mean","max", "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená)  # Subset jen říká, kam chci barvu

data[data["plocha"] != -1].groupby(["město", "okres"])["cena_metr"].agg(["min",  np.median, "mean","max", "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená)  # Subset jen říká, kam chci barvu

#  sort_values( ascending = False, "median").head(15)  mi nahrazuje    .nsmalest(15, "median"),  protože jsem chtěl aby to bylo seřazené v sestupném pořadí


### 2.c) Ceny podle okresů

### 2.c.1) Nejdražší okresy

In [ ]:
data.groupby("okres")["cena"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)

data[data["plocha"] != -1].groupby("okres")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)


### 2.c.2) Nejlevnější okresy

In [ ]:
data.groupby("okres")["cena"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená) 

data[data["plocha"] != -1].groupby("okres")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená) 


### 2.d) Ceny podle krajů

In [ ]:
data.groupby("kraj")["cena"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = rudá)

data[data["plocha"] != -1].groupby("kraj")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = rudá)


### 3) Ceny podle místnosí PRAHA a MOST


### 3.a) Ceny podle místností PRAHA

In [ ]:
data[data["město"] == "Praha"].groupby("pokoje")["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

data[(data["město"] == "Praha") & (data["plocha"] != -1)].groupby("pokoje")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)



# Postřeh: kde je větší nabídka, jsou i větší extrémy, aspoň co se  minim týče

## Postřeh: V praze byt za 320.000 a za 180.000 = Ve skutečnosti jen zprostředkování nájmu :/
# Vyškrtnuto

### 3.b) Ceny městských částí Prahy
* celkem 95 městských částí

### 3.b.1) Nejdražší pražské části

In [ ]:
data[data["město"] == "Praha"].groupby("oblast")["cena"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, columns = "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)


data[(data["město"] == "Praha") & (data["plocha"] != -1)].groupby("oblast")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).nlargest(15, columns = "median").apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = červená)


### 3.b.2) Nejlevnější pražské části

In [ ]:
data[data["město"] == "Praha"].groupby("oblast")["cena"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená)

data[(data["město"] == "Praha") & (data["plocha"] != -1)].groupby("oblast")["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).sort_values( "median",ascending = False).tail(15).apply(lambda x: round(x)).style.bar(subset = ["mean", "median"], align = "mid", color = zelená)


### 3.c) Ceny podle místností MOST

In [ ]:
#data[data["město"] == "Most"].groupby(["město", "pokoje"])["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

#Ten poslední jediný záznam to trošku rozbourává ...Takže ho zkusím spíš odebrat
data[(data["město"] == "Most") & (data["pokoje"] != "4+kk")].groupby(["město", "pokoje"])["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

data[(data["plocha"] != -1) &(data["město"] == "Most") & (data["pokoje"] != "4+kk")].groupby(["město", "pokoje"])["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)



### 3.d)  PRAHA vs MOST

In [ ]:
# data[(data.město == "Praha") | (data.město == "Most") ].groupby(["pokoje","město" ])["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)

# Porovnám jen ty pokoje, které existují i v mostě, vyhnu se tím super extrémům
# Odebral jsem i 4+kk protože za Most byl jediný záznam - viz předešlý graf

data[((data.město == "Praha") | (data.město == "Most")) & (data.pokoje != "4+kk")& (data.pokoje != "5+1") & (data.pokoje != "5+kk") & (data.pokoje != "6-a-vice") & (data.pokoje != "atypicky") & (data.pokoje != "5+1") ]\
.groupby(["pokoje","město" ])["cena"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)


data[(data["plocha"] != -1) & ((data.město == "Praha") | (data.město == "Most")) & (data.pokoje != "4+kk")& (data.pokoje != "5+1") & (data.pokoje != "5+kk") & (data.pokoje != "6-a-vice") & (data.pokoje != "atypicky") & (data.pokoje != "5+1") ]\
.groupby(["pokoje","město" ])["cena_metr"].agg(['min', "mean", np.median,'max', "count"]).apply(lambda x: round(x)).style.bar(subset = ["min", "mean", "median", "max"], align = "mid", color = rudá)


#data[data["plocha"] != -1]

### Zajímavý extrém... Praha 1+kk za 10 mega

In [ ]:
data[(data.město == "Praha") & (data.cena == 10390)  & (data.pokoje == "1+kk")]

### 4) Pivoty

### 4.a) Pivot Pražské části x Pokoje
* 95 pražských částí

In [ ]:
table = pd.pivot_table(data[data["město"] == "Praha"], values='cena', index=['oblast'],
                    columns=['pokoje'], aggfunc=np.mean).apply(lambda x: round(x))

# V1- Jen bílé nuly   # table.highlight_null(null_color='white')
# V2 - cmap = cm -> bílo-zelená    cm = sns.light_palette("green", as_cmap=True)
# V3 - cmap = "viridis"
# V4 - cmap = "magma"
# V5 - Inferno
# V6 - hot

# V5 - "magma_r"
table.style.background_gradient(cmap='magma_r')

### Nedodělaný HeatMap

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,35)) 

sns.heatmap(table, annot=True,fmt=".0f")

### 4.b) Pivot Kraje x Pokoje

In [ ]:
table = pd.pivot_table(data, values='cena', index=['kraj'],
                    columns=['pokoje'], aggfunc=np.mean).apply(lambda x: round(x))

cm = sns.light_palette("green", as_cmap=True)
table.style.background_gradient(cmap='magma_r')



### 4.c) Pivot Kraje (bez Prahy) x Pokoje

In [ ]:
table = pd.pivot_table(data[data.město != "Praha"], values='cena', index=['kraj'],
                    columns=['pokoje'], aggfunc=np.mean).apply(lambda x: round(x))

cm = sns.light_palette("green", as_cmap=True)
table.style.background_gradient(cmap='magma_r')

In [ ]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "8pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "24pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '600px'),
                        ('font-size', '36pt')])
]

In [ ]:
table.style.background_gradient(cmap = "magma_r")\
    .set_properties(**{'max-width': '1200px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .set_precision(10)\
    .set_table_styles(magnify())

# Precision ovlivní délku čísla

## 5) Interaktívní widgety = DODĚLAT

In [ ]:
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

cities = widgets.Dropdown(options =  unique_sorted_values_plus_ALL(data.city_one_praha))

def dropdown_year_eventhandler(change):
    display(data[data.city_one_praha == change.new])

cities.observe(dropdown_year_eventhandler, names='value')

display(cities)

 

In [ ]:
output = widgets.Output()
display(output)

In [ ]:
from ipywidgets import interact

import jupyterlab-widgets as lab_widgets

In [ ]:
#    jupyter labextension install @jupyter-widgets/jupyterlab-manager
# Nevidím nikde svj návod, ale jel jsme jinak podle: https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916



ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique
################################

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(data.city_one_praha))

output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()
    with output_year:
        if (change.new == ALL):
            display(data)
        else:
            display(data[data.city_one_praha == change.new])

dropdown_year.observe(dropdown_year_eventhandler, names='value')

display(dropdown_year)

        





In [ ]:
display(output_year)
